In [2]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import os

In [3]:
'''
Read in saved dataset to use to feed into the model.
Further pre-processing still needs to be done on the state vectors
'''

data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/opponent_dataset.pkl')


data_df.head()

,season,gameId,playId,playIndex,timeIndex,time,playerId,football_pos,state,next_state,reward,action,next_action
0,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,38707.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[44.11, 24.92, 1.01, 10.05, 0.76, 299.74, 292...","[[43.17, 25.29, 1.01, 10.09, 0.75, 296.5, 291....",[-0.09],"[[0.94, -0.37, 0.0, -0.04, 0.01, 3.24, 0.45, 0...","[[0.87, -0.3, -0.01, -0.04, -0.05, -1.93, 1.07..."
1,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,40191.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[43.9, 21.75, 0.91, 9.11, 1.06, 283.77, 289.8...","[[43.03, 22.05, 0.92, 9.15, 1.11, 285.7, 288.8...",[-0.09],"[[0.87, -0.3, -0.01, -0.04, -0.05, -1.93, 1.07...","[[0.92, -0.01, -0.01, 0.01, -0.1, 0.0, -0.08, ..."
2,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,42450.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[45.07, 49.79, 0.91, 9.18, 0.44, 251.62, 270....","[[44.15, 49.8, 0.92, 9.17, 0.54, 251.62, 270.1...",[-0.09],"[[0.92, -0.01, -0.01, 0.01, -0.1, 0.0, -0.08, ...","[[0.69, -0.08, 0.0, 0.03, 0.05, -0.69, 0.69, 0..."
3,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,43306.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[50.5, 46.24, 0.7, 7.0, 1.05, 275.39, 277.13,...","[[49.81, 46.32, 0.7, 6.97, 1.0, 276.08, 276.44...",[-0.09],"[[0.69, -0.08, 0.0, 0.03, 0.05, -0.69, 0.69, 0...","[[0.85, -0.29, 0.01, 0.12, -0.08, 2.19, -0.11,..."
4,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,46263.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[41.11, 32.25, 0.91, 9.01, 1.72, 292.28, 288....","[[40.26, 32.54, 0.9, 8.89, 1.8, 290.09, 288.69...",[-0.09],"[[0.85, -0.29, 0.01, 0.12, -0.08, 2.19, -0.11,...","[[0.85, -0.29, 0.01, 0.05, -0.18, -3.12, -1.75..."


In [4]:
'''
State vector 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''

data_df.loc[0,'state'][0]

array([ 44.11      ,  24.92      ,   1.01      ,  10.05      ,
         0.76      , 299.74      , 292.21      ,  44.11      ,
        -1.73      , 209.74      , 202.21      ,   0.6792262 ,
        -0.733929  ,   0.91197393,   0.41024816,  36.997     ,
         2.715     ,   5.042     ])

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [6]:
'''
Create torch dataset
'''

features = torch.zeros(len(data_df)).to(device)
labels = torch.zeros(len(data_df)).to(device)

print("size = ", features.size())

size =  torch.Size([215429])


In [7]:
'''
Function that splits the data into a training, validation, and test set
'''
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)

print(test_data.columns)

print(f"Length of training data: {len(train_data)}")

Index(['season', 'gameId', 'playId', 'playIndex', 'timeIndex', 'time',
       'playerId', 'football_pos', 'state', 'next_state', 'reward', 'action',
       'next_action'],
      dtype='object')
Length of training data: 150800


In [8]:
'''
Predict ['s', 'a', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']
from 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''



'''
Define hyperparamters
'''
BATCH_SIZE = 64
state_size = len(data_df.loc[0,'state'][0])
action_size = 8
gamma = 0.99
#max_action = torch.tensor(max_action, dtype=torch.float32).to(device=device)

print(f"State size: {state_size}")
print(f"Action size: {action_size}")
    
'''
First CNN model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Conv2d(12, 24, (6, 1)),
    nn.LeakyReLU(),
    nn.Conv2d(24, 48, (1,1)),
    nn.Flatten(),
    nn.Linear(48*6*6,action_size)
)

cnn_model.to(device=device)
'''

State size: 18
Action size: 8


'\nFirst CNN model\ncnn_model = nn.Sequential(\n    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),\n    nn.LeakyReLU(),\n    nn.Conv2d(12, 24, (6, 1)),\n    nn.LeakyReLU(),\n    nn.Conv2d(24, 48, (1,1)),\n    nn.Flatten(),\n    nn.Linear(48*6*6,action_size)\n)\n\ncnn_model.to(device=device)\n'

In [8]:
'''
cnn_model = nn.Sequential(
    nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(64,128, (2,2), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(128,256, (1,1), stride=1,padding=0),
    #nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Flatten(),
    nn.Linear(4352, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 100),
    nn.LeakyReLU(),
    nn.Linear(100, action_size)
)
cnn_model.to(device=device)
'''

# 0.3046 val accuracy after 4 epochs, batch=64
cnn_model = nn.Sequential(
    nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
    nn.Flatten(),
    nn.Linear(924,300), 
    nn.LeakyReLU(),
    nn.Linear(300,100), 
    nn.LeakyReLU(),
    nn.Linear(100,action_size)
)


'''
cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,100), 
    nn.LeakyReLU(),
    nn.Linear(100,50), 
    nn.LeakyReLU(),
    nn.Linear(50,action_size)
)

cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,60), 
    nn.Dropout(p=0.5),
    nn.LeakyReLU(),
    nn.Linear(60,action_size)
)
'''


cnn_model.to(device=device)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(2, 2), stride=(1, 1))
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=924, out_features=300, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=300, out_features=100, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=100, out_features=8, bias=True)
)

In [9]:
# define loss function
mse_loss_fn = nn.MSELoss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters())

In [10]:
np.stack(test_data.iloc[0,:]['action'])[0,1:3]

array([-0.21, -0.02])

In [9]:
'''
Given indices, train model on batch
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''
def train_loop(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    numpy_state_input = numpy_state.reshape(batch_size, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
    
    true_state = torch.tensor(numpy_state, dtype=torch.float32).to(device=device)
    true_action = torch.tensor(np.stack(play['action'].values)[:,0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    #true_state_mod = true_state.reshape(batch_size, -1, len(true_action))
    true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_action = model(true_state_mod)
    
    loss = loss_fn(predicted_action, true_action)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [10]:
def test_loop(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            true_state = torch.tensor(play['state'], dtype=torch.float32).to(device=device)
            true_action = torch.tensor(play['action'][0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
            
            # useful for if testing in batches
            #true_state = torch.tensor(np.stack(play['state'].values), dtype=torch.float32).to(device=device)
            #true_action = torch.tensor(np.stack(play['action'].values), dtype=torch.float32).to(device=device)

            numpy_state_input = play['state'].reshape(1, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
                           
            true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_action = model(true_state_mod)
  
            true_action = torch.unsqueeze(true_action, 0)

            loss = loss_fn(predicted_action, true_action)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [13]:
import random

epochs = 4
break_var = False
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/2) == 0): #and (i != 0):
            val_loss = test_loop(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
            
torch.save(cnn_model.state_dict(), 'opponent_model.pt')

Num batches: 2357
At epoch 0, iter 0: train loss = 0.014009935781359673
At epoch 0, iter 0: val loss = 2.1975460052490234
At epoch 0, iter 1178: train loss = 0.005030847154557705
At epoch 0, iter 1178: val loss = 0.29499542713165283
At epoch 0, iter 2356: train loss = 0.017518991604447365
At epoch 0, iter 2356: val loss = 0.2860490083694458
At epoch 1, iter 0: train loss = 0.004267571028321981
At epoch 1, iter 0: val loss = 0.2881411910057068
At epoch 1, iter 1178: train loss = 0.004351135343313217
At epoch 1, iter 1178: val loss = 0.28011730313301086
At epoch 1, iter 2356: train loss = 0.017322488129138947
At epoch 1, iter 2356: val loss = 0.27796420454978943
At epoch 2, iter 0: train loss = 0.0042358096688985825
At epoch 2, iter 0: val loss = 0.27904003858566284
At epoch 2, iter 1178: train loss = 0.004359383136034012
At epoch 2, iter 1178: val loss = 0.27592310309410095
At epoch 2, iter 2356: train loss = 0.016780538484454155
At epoch 2, iter 2356: val loss = 0.2726179361343384
At e

In [11]:

teammate_model = nn.Sequential(
    nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
    nn.Flatten(),
    nn.Linear(924,300), 
    nn.LeakyReLU(),
    nn.Linear(300,100), 
    nn.LeakyReLU(),
    nn.Linear(100,action_size)
)

opponent_model = nn.Sequential(
    nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
    nn.Flatten(),
    nn.Linear(924,300), 
    nn.LeakyReLU(),
    nn.Linear(300,100), 
    nn.LeakyReLU(),
    nn.Linear(100,action_size)
)

teammate_model.to(device)
opponent_model.to(device)

teammate_model.load_state_dict(torch.load("teammate_model.pt"))
opponent_model.load_state_dict(torch.load("opponent_model.pt"))

<All keys matched successfully>

In [12]:
teammate_loss = test_loop(val_data, teammate_model, F.mse_loss, device)
opponent_loss = test_loop(val_data, opponent_model, F.mse_loss, device)

print(teammate_loss)
print(opponent_loss)

tensor(0.3003, device='cuda:0')
tensor(0.2730, device='cuda:0')
